### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [4]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies = ratings_mat.shape[1]# number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))# total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies

    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i,k])

        # print results for iteration
        print("%d \t\t %f" % (iteration + 1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features = 4, learning_rate=0.005, iters = 10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 45.031541
2 		 16.201612
3 		 4.376275
4 		 2.854238
5 		 2.715358
6 		 2.672593
7 		 2.640668
8 		 2.609807
9 		 2.577605
10 		 2.543038


In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.776456     8.95410008  10.38770423  10.47339218]
 [  8.79066821   7.38401521   9.059608     8.75435582]
 [  7.95547985   6.89499669   8.52119474   7.50476569]
 [  9.39024379   8.28157534   9.88195191   9.81875491]
 [  8.8742193    7.35513406   9.15145094   8.23943124]
 [  6.9100822    6.02578348   7.15310625   6.85341875]
 [  9.34291694   7.81973522   9.74090694   9.18939591]
 [  8.61790921   6.91012489   8.67082348   8.05658644]
 [  8.2074557    7.49563082   8.81065071   8.6877005 ]
 [  7.89255313   6.59296202   8.00177622   7.6722161 ]
 [  8.84620732   7.8236755    9.22779951   8.93321116]
 [  9.38723155   8.37500095  10.03610026   9.73467842]
 [  9.35328639   8.1001201    9.62861262   9.31338908]
 [  6.26126823   5.86319845   6.75415861   6.99040638]
 [  9.56862282   8.75033329  10.29328446   9.66477157]
 [  9.80767669   8.55198322  10.12110729   9.79026634]
 [  8.33860259   7.52596667   8.90419293   7.94021838]
 [  7.24871819   5.98827484   7.30305318   7.28273172]
 [  8.1897

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [7]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate = 0.005, iters = 250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 41.204937
2 		 13.216395
3 		 3.749839
4 		 2.775755
5 		 2.667562
6 		 2.620850
7 		 2.582586
8 		 2.543151
9 		 2.499958
10 		 2.452006
11 		 2.398622
12 		 2.339223
13 		 2.273315
14 		 2.200532
15 		 2.120688
16 		 2.033824
17 		 1.940253
18 		 1.840581
19 		 1.735711
20 		 1.626818
21 		 1.515290
22 		 1.402646
23 		 1.290437
24 		 1.180144
25 		 1.073094
26 		 0.970399
27 		 0.872929
28 		 0.781307
29 		 0.695934
30 		 0.617020
31 		 0.544615
32 		 0.478646
33 		 0.418944
34 		 0.365259
35 		 0.317286
36 		 0.274672
37 		 0.237037
38 		 0.203980
39 		 0.175092
40 		 0.149970
41 		 0.128218
42 		 0.109460
43 		 0.093342
44 		 0.079537
45 		 0.067745
46 		 0.057696
47 		 0.049148
48 		 0.041887
49 		 0.035726
50 		 0.030500
51 		 0.026069
52 		 0.022312
53 		 0.019123
54 		 0.016416
55 		 0.014114
56 		 0.012155
57 		 0.010485
58 		 0.009060
59 		 0.007841
60 		 0.006796
61 		 0.005900
62 		 0.005129
63 		 0.004465
64 		

In [8]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.          10.          10.          10.00000001]
 [  9.9999993    3.99999955   8.99999967  10.00000091]
 [  7.99999835   8.99999898   9.99999927   5.00000209]
 [  8.9999989    7.99999934   9.99999955  10.0000014 ]
 [  9.99999936   4.99999964   8.99999977   9.00000082]
 [  6.00000072   4.00000046  10.00000036   5.99999914]
 [  9.00000048   8.00000033  10.00000027   8.99999941]
 [ 10.00000149   5.00000095   9.00000069   7.99999814]
 [  7.00000169   8.00000105  10.00000076   7.99999789]
 [  8.99999986   4.9999999    8.99999992   7.00000017]
 [  8.99999958   7.99999972   9.99999979   8.00000055]
 [  9.00000029  10.0000002   10.00000014   8.99999962]
 [ 10.00000023   9.00000013  10.00000009   7.99999972]
 [  5.00000072   8.00000044   5.00000032   7.99999909]
 [  9.99999948   7.99999966   9.99999974  10.00000065]
 [  8.99999972   8.99999983   9.99999987  10.00000034]
 [  9.00000041   8.00000026   8.00000018   7.99999948]
 [ 10.           8.           1.          10.        ]
 [  4.9999

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [9]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [10]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features = 4, learning_rate = 0.005, iters = 250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 43.403165
2 		 15.169108
3 		 4.274929
4 		 2.949565
5 		 2.830023
6 		 2.800074
7 		 2.783177
8 		 2.769398
9 		 2.756390
10 		 2.743346
11 		 2.729804
12 		 2.715383
13 		 2.699725
14 		 2.682483
15 		 2.663310
16 		 2.641866
17 		 2.617816
18 		 2.590834
19 		 2.560615
20 		 2.526887
21 		 2.489423
22 		 2.448064
23 		 2.402742
24 		 2.353498
25 		 2.300503
26 		 2.244069
27 		 2.184658
28 		 2.122867
29 		 2.059410
30 		 1.995077
31 		 1.930687
32 		 1.867035
33 		 1.804838
34 		 1.744695
35 		 1.687054
36 		 1.632208
37 		 1.580297
38 		 1.531333
39 		 1.485225
40 		 1.441814
41 		 1.400905
42 		 1.362284
43 		 1.325745
44 		 1.291094
45 		 1.258157
46 		 1.226784
47 		 1.196843
48 		 1.168219
49 		 1.140812
50 		 1.114534
51 		 1.089305
52 		 1.065052
53 		 1.041708
54 		 1.019210
55 		 0.997498
56 		 0.976516
57 		 0.956210
58 		 0.936526
59 		 0.917413
60 		 0.898819
61 		 0.880693
62 		 0.862980
63 		 0.845627
64 		

In [11]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.034229143638841:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [12]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features = 4, learning_rate = 0.005, iters = 20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.157189
2 		 10.750311
3 		 7.379793
4 		 5.689174
5 		 4.624743
6 		 3.883530
7 		 3.334851
8 		 2.911362
9 		 2.574601
10 		 2.300959
11 		 2.075044
12 		 1.886253
13 		 1.726920
14 		 1.591285
15 		 1.474909
16 		 1.374311
17 		 1.286737
18 		 1.209991
19 		 1.142319
20 		 1.082310


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [13]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [14]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
